<a href="https://colab.research.google.com/github/LegendSeyi/Analysing-Electricity-Usage-Pattern-In-Rural-Areas-In-Nigeria-Using-Unsupervised-ML/blob/main/Analysing_Electricity_Usage_Patterning_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analysing Electricity Usage Pattern In Rural Areas In Nigeria Using Unsupervised ML

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [45]:
url = 'https://raw.githubusercontent.com/LegendSeyi/Dataset/refs/heads/main/nigeria_rural_electricity_usage.csv'

pd.set_option('display.max_columns', None)
data = pd.read_csv(url, encoding= 'utf-8')

In [46]:
data

,timestamp,state,community,household_id,energy_kWh,source,weather,appliance_usage_level,avg_voltage,blackout_hours
0,2025-03-01,Lagos,Agbowa,H5264,7.99,Grid,Rainy,Medium,210.3,3.5
1,2025-03-20,Oyo,Igboora,H4998,9.08,Grid,Rainy,Medium,205.6,5.1
2,2025-01-05,Lagos,Epe,H6356,7.25,Grid,Sunny,Low,180.5,2.9
3,2025-01-15,Ogun,Ijebu,H6182,3.70,Solar,Sunny,Low,244.3,3.5
4,2025-03-27,Lagos,Agbowa,H7931,2.46,Grid,Sunny,Medium,192.0,4.8
...,...,...,...,...,...,...,...,...,...,...
1105,2025-03-06,Ogun,Ifo,H8076,2.60,Solar,Sunny,High,203.7,1.0
1106,2025-02-06,Lagos,Epe,H1354,3.71,Grid,Cloudy,Medium,216.4,0.5
1107,2025-02-24,Lagos,Badagry,H4245,9.15,Grid,Cloudy,Medium,207.0,2.5
1108,2025-01-24,Oyo,Ogbomoso,H2784,1.05,Generator,Rainy,High,212.9,5.2


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              1110 non-null   object 
 1   state                  1110 non-null   object 
 2   community              1110 non-null   object 
 3   household_id           1110 non-null   object 
 4   energy_kWh             1110 non-null   float64
 5   source                 1110 non-null   object 
 6   weather                1110 non-null   object 
 7   appliance_usage_level  1110 non-null   object 
 8   avg_voltage            1110 non-null   float64
 9   blackout_hours         1110 non-null   float64
dtypes: float64(3), object(7)
memory usage: 86.8+ KB


In [48]:
data.head()

,timestamp,state,community,household_id,energy_kWh,source,weather,appliance_usage_level,avg_voltage,blackout_hours
0,2025-03-01,Lagos,Agbowa,H5264,7.99,Grid,Rainy,Medium,210.3,3.5
1,2025-03-20,Oyo,Igboora,H4998,9.08,Grid,Rainy,Medium,205.6,5.1
2,2025-01-05,Lagos,Epe,H6356,7.25,Grid,Sunny,Low,180.5,2.9
3,2025-01-15,Ogun,Ijebu,H6182,3.70,Solar,Sunny,Low,244.3,3.5
4,2025-03-27,Lagos,Agbowa,H7931,2.46,Grid,Sunny,Medium,192.0,4.8


##DATA CLEANING

In [49]:
data.shape

(1110, 10)

In [50]:
data.columns

Index(['timestamp', 'state', 'community', 'household_id', 'energy_kWh',
       'source', 'weather', 'appliance_usage_level', 'avg_voltage',
       'blackout_hours'],
      dtype='object')

In [51]:
data.nunique()

,0
timestamp,90
state,3
community,12
household_id,1050
energy_kWh,662
source,3
weather,3
appliance_usage_level,3
avg_voltage,577
blackout_hours,94


In [52]:
data['household_id'].unique()

array(['H5264', 'H4998', 'H6356', ..., 'H4245', 'H2784', 'H2402'],
      dtype=object)

Removing the "Houshold_id" column,it is not useful for this project

In [53]:
data['household_id'].count()

np.int64(1110)

In [54]:
data.drop('household_id', axis=1, inplace=True)

In [55]:
data.head()

,timestamp,state,community,energy_kWh,source,weather,appliance_usage_level,avg_voltage,blackout_hours
0,2025-03-01,Lagos,Agbowa,7.99,Grid,Rainy,Medium,210.3,3.5
1,2025-03-20,Oyo,Igboora,9.08,Grid,Rainy,Medium,205.6,5.1
2,2025-01-05,Lagos,Epe,7.25,Grid,Sunny,Low,180.5,2.9
3,2025-01-15,Ogun,Ijebu,3.70,Solar,Sunny,Low,244.3,3.5
4,2025-03-27,Lagos,Agbowa,2.46,Grid,Sunny,Medium,192.0,4.8


#### timestamp column

In [56]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [57]:
data['timestamp'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1110 entries, 0 to 1109
Series name: timestamp
Non-Null Count  Dtype         
--------------  -----         
1110 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 8.8 KB


##### sorting the data according to the timestamp in ascending order

In [58]:
data_sorted = data.sort_values(by='timestamp', ascending=True)

In [59]:
data_sorted

,timestamp,state,community,energy_kWh,source,weather,appliance_usage_level,avg_voltage,blackout_hours
125,2025-01-01,Oyo,Ogbomoso,3.58,Solar,Rainy,Low,208.3,7.3
1078,2025-01-01,Ogun,Sagamu,6.52,Solar,Cloudy,Medium,207.7,0.7
1068,2025-01-01,Oyo,Oyo Town,6.13,Grid,Cloudy,Low,171.1,7.2
819,2025-01-01,Ogun,Sagamu,5.24,Generator,Sunny,High,215.9,5.1
894,2025-01-01,Ogun,Ijebu,3.65,Generator,Sunny,Medium,193.2,3.5
...,...,...,...,...,...,...,...,...,...
627,2025-03-31,Oyo,Iseyin,2.82,Grid,Sunny,Low,193.6,7.8
184,2025-03-31,Lagos,Badagry,4.21,Grid,Cloudy,Medium,219.9,2.9
639,2025-03-31,Ogun,Ijebu,4.56,Generator,Rainy,Medium,190.9,1.8
396,2025-03-31,Ogun,Ifo,3.34,Grid,Sunny,Low,204.5,4.3


In [60]:
data_sorted['timestamp'].nunique()

90

the data recollection spanned for 90 days in the year 2025

## State Column

In [63]:
data_sorted.state.unique()

array(['Oyo', 'Ogun', 'Lagos'], dtype=object)

data was collected from 3 states in Nigeria which are: OYO, OGUN, LAGOS

In [69]:
data_sorted.state.info()

<class 'pandas.core.series.Series'>
Index: 1110 entries, 125 to 322
Series name: state
Non-Null Count  Dtype 
--------------  ----- 
1110 non-null   object
dtypes: object(1)
memory usage: 17.3+ KB


### Community Column

In [71]:
data_sorted.community.head()

,community
125,Ogbomoso
1078,Sagamu
1068,Oyo Town
819,Sagamu
894,Ijebu


In [72]:
data_sorted.community.nunique()

12

12 different community from the 3 states

In [73]:
state_community_counts = data_sorted.groupby('state')['community'].nunique()

In [74]:
state_community_counts

,community
state,
Lagos,4
Ogun,4
Oyo,4


4 communities in each states

In [80]:
data_sorted.loc[data_sorted['state'] == 'Lagos', 'community'].unique()

array(['Epe', 'Ikorodu', 'Badagry', 'Agbowa'], dtype=object)

Rural Communities under lagos are Epe, Ikorodu, Badagry, Agbowa

In [81]:
data_sorted.loc[data_sorted['state'] == 'Oyo', 'community'].unique()

array(['Ogbomoso', 'Oyo Town', 'Iseyin', 'Igboora'], dtype=object)

Rural Communities under Oyo are Ogbomoso, Oyo Town, Iseyin, Igboora

In [82]:
data_sorted.loc[data_sorted['state'] == 'Ogun', 'community'].unique()

array(['Sagamu', 'Ijebu', 'Ifo', 'Ayetoro'], dtype=object)

Rural Communities under Ogun are 'Sagamu', 'Ijebu', 'Ifo', 'Ayetoro

In [100]:
print(f'lagos data: {data_sorted.loc[data_sorted['state'] == 'Lagos', 'community'].count()}')

print(f'Oyo data: {data_sorted.loc[data_sorted['state'] == 'Oyo', 'community'].count()}')

print(f'Ogun data: {data_sorted.loc[data_sorted['state'] == 'Ogun', 'community'].count()}')

lagos data: 357
Oyo data: 375
Ogun data: 378


### energy_kWh column

In [105]:
data_sorted.energy_kWh.head()

,energy_kWh
125,3.58
1078,6.52
1068,6.13
819,5.24
894,3.65


In [106]:
data_sorted.energy_kWh.describe()

,energy_kWh
count,1110.000000
mean,5.519757
std,2.827698
min,0.600000
25%,3.322500
50%,5.010000
75%,7.250000
max,12.000000


In [107]:
data_sorted.energy_kWh.max()

12.0